In [2]:
import zipfile
import os

zip_file = "dataset.zip"

extract_path = "/content/dataset"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Arhiva a fost dezarhivata cu succes.")

Arhiva a fost dezarhivata cu succes.


In [5]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image

# Setări
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
DATA_DIR = "/content/dataset/SkinMate-Dataset"
ALLOWED_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}

# 1. Verificarea și conversia imaginilor
def verify_and_convert_images(root_dir):
    for root, _, files in os.walk(root_dir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            img_path = os.path.join(root, file)

            if ext not in ALLOWED_EXT:
                print(f"Fișier invalid eliminat: {img_path}")
                os.remove(img_path)  # Elimină fișierul invalid
            else:
                try:
                    img = Image.open(img_path)
                    img = img.convert("RGB")  # Asigură formatul RGB
                    img.save(img_path)
                except Exception as e:
                    print(f"Eroare la conversie: {img_path} - {e}")

verify_and_convert_images(DATA_DIR)

# 2. Încărcarea datasetului (păstrăm class_names înainte de preprocesare)
raw_train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

raw_val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = raw_train_ds.class_names
print("Classes:", class_names)

# 3. Normalizarea și prefetching-ul datelor
AUTOTUNE = tf.data.AUTOTUNE
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = raw_train_ds.map(lambda x, y: (normalization_layer(x), y)).prefetch(buffer_size=AUTOTUNE)
val_ds = raw_val_ds.map(lambda x, y: (normalization_layer(x), y)).prefetch(buffer_size=AUTOTUNE)

# 4. Crearea modelului CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# 5. Antrenarea modelului
epochs = 10
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Found 1000 files belonging to 4 classes.
Using 800 files for training.
Found 1000 files belonging to 4 classes.
Using 200 files for validation.
Classes: ['acnes', 'blackheads', 'darkspots', 'wrinkles']
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.3911 - loss: 1.9822 - val_accuracy: 0.7400 - val_loss: 0.6686
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.7163 - loss: 0.6294 - val_accuracy: 0.7450 - val_loss: 0.5614
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.8192 - loss: 0.4712 - val_accuracy: 0.8400 - val_loss: 0.3757
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.8769 - loss: 0.2939 - val_accuracy: 0.8400 - val_loss: 0.4656
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9180 - loss: 0.2629 - val_accuracy: 0.8650 - val_loss: 0.3045
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.9462 - loss: 0.1595 - val_accuracy: 0.8200 - val_loss: 0.5787
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 

In [ ]:
import pandas as pd

# functie pentru formatarea pretului
def format_price(price_val):
    if isinstance(price_val, str):
        if 'Rp' in price_val:
            return price_val.strip()
        else:
            try:
                price_num = float(price_val.replace('.', '').replace(',', '.'))
                return f"Rp {price_num:,.0f}".replace(',', '.')
            except:
                return price_val.strip()
    else:
        try:
            price_num = float(price_val)
            return f"Rp {price_num:,.0f}".replace(',', '.')
        except:
            return str(price_val)
        
# functie pentru recomandare de produse
def get_product_recommendations_from_excel(problem_tag, excel_path='Dataset Skincare Product.xlsx'):
    df = pd.read_excel(excel_path, sheet_name='skinTreatment')

    df = df.fillna('')
    df['Tags'] = df['Tags'].astype(str).str.lower()

    recomandari = df[df['Tags'].str.contains(problem_tag.lower(), na=False)]

    if recomandari.empty:
        return ["Nu există produse disponibile pentru această problemă."]

    produse = []
    for _, row in recomandari.iterrows():
        price = format_price(row['Price'])
        info = (
            f"🔹 {row['Brand']} - {row['Product Name']}\n"
            f"   💰 {price}\n"
            f"   🔗 {row['Links']}\n"
        )
        produse.append(info)

    return produse